In [1]:
from collections import namedtuple
import scipy.stats as sps
import statsmodels.stats.api as sms
from tqdm.notebook import tqdm as tqdm_notebook # tqdm – библиотека для визуализации прогресса в цикле
from collections import defaultdict
from statsmodels.stats.proportion import proportion_confint
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, palette='Set2')
ExperimentComparisonResults = namedtuple('ExperimentComparisonResults', 
                                        ['pvalue', 'effect', 'ci_length', 'left_bound', 'right_bound'])

# Абсолютный t-test критерий

In [2]:
# 2. Создание тестируемого критерия
def absolute_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    var_mean_control  = np.var(control) / len(control)
    var_mean_test  = np.var(test) / len(test)
    
    difference_mean = mean_test - mean_control
    difference_mean_var = var_mean_control + var_mean_test
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([0.025, 0.975])
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

AA-тест.

In [3]:
# 3. Заводим счетчик
bad_cnt = 0

# 4. Цикл проверки
N = 100000
for i in tqdm_notebook(range(N)):
    # 4.a. Тестирую AA - тест
    control = sps.expon(scale=1000).rvs(500)
    test = sps.expon(scale=1000).rvs(600)

    # 4.b. Запускаю критерий
    _, _, _, left_bound, right_bound = absolute_ttest(control, test)
    
    # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале
    if left_bound > 0 or right_bound < 0:
        bad_cnt += 1

# 5. Строю доверительный интервал для конверсии ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')
# Результат
print(f"Реальный уровень значимости: {round(bad_cnt / N, 4)};"
      f" доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]")


Реальный уровень значимости: 0.0511; доверительный интервал: [0.0497, 0.0525]


# Относительный t-test критерий

## Корректный критерий

AB-проверка

In [4]:
# 2. Создание тестируемого критерия
def relative_ttest(control, test):
    mean_control = np.mean(control)
    var_mean_control  = np.var(control) / len(control)

    difference_mean = np.mean(test) - mean_control
    difference_mean_var  = np.var(test) / len(test) + var_mean_control
    
    covariance = -var_mean_control

    relative_mu = difference_mean / mean_control
    relative_var = difference_mean_var / (mean_control ** 2) \
                    + var_mean_control * ((difference_mean ** 2) / (mean_control ** 4))\
                    - 2 * (difference_mean / (mean_control ** 3)) * covariance
    relative_distribution = sps.norm(loc=relative_mu, scale=np.sqrt(relative_var))
    left_bound, right_bound = relative_distribution.ppf([0.025, 0.975])
    
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(relative_distribution.cdf(0), relative_distribution.sf(0))
    effect = relative_mu
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

In [5]:
# 3. Заводим счетчик
bad_cnt = 0

# 4. Цикл проверки
N = 100000
for i in tqdm_notebook(range(N)):
    # 4.a. Тестирую AB - тест
    control = sps.expon(scale=1000).rvs(2000)
    test = sps.expon(scale=1000).rvs(2100)
    test *= 2

    # 4.b. Запускаю критерий
    _, _, _, left_bound, right_bound = relative_ttest(control, test)
    
    # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале
    if left_bound > 1 or right_bound < 1:
        bad_cnt += 1

# 5. Строю доверительный интервал для конверсии ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')
# Результат
print(f"Реальный уровень значимости: {round(bad_cnt / N, 4)};"
      f" доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]")



Реальный уровень значимости: 0.0501; доверительный интервал: [0.0487, 0.0514]


## Некорректный критерий

In [6]:
# 2. Создание тестируемого критерия
def relative_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    var_mean_control  = np.var(control) / len(control)
    var_mean_test  = np.var(test) / len(test)

    difference_mean = mean_test - mean_control
    difference_mean_var = var_mean_control + var_mean_test
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([0.025, 0.975])
    left_bound = left_bound / np.mean(control)   # Деление на среднее
    right_bound = right_bound / np.mean(control) # Деление на среднее

    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

AB - проверка

In [7]:
# 3. Заводим счетчик
bad_cnt = 0

# 4. Цикл проверки
N = 100000
for i in tqdm_notebook(range(N)):
    # 4.a. Тестирую AB - тест
    control = sps.expon(scale=1000).rvs(1000)
    test = sps.expon(scale=1000).rvs(1100)
    test *= 2

    # 4.b. Запускаю критерий
    _, _, _, left_bound, right_bound = relative_ttest(control, test)
    
    # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале
    if left_bound > 1 or right_bound < 1:
        bad_cnt += 1

# 5. Строю доверительный интервал для конверсии ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')
# Результат
print(f"Реальный уровень значимости: {round(bad_cnt / N, 4)};"
      f" доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]")


Реальный уровень значимости: 0.1278; доверительный интервал: [0.1258, 0.1299]
